# Code to Generate Biochar Atomistic Models

<img src="biochar.png" width="450" height="450">


## Open Pymol 

In [1]:
# Import pymol
from ipymol import viewer as pymol

In [2]:
# Initiate PyMol RPC module
import glob,os,sys,subprocess,random, math
from pymol_rpc import pymol_jupyter_builder
import numpy as np
import random
from scipy.optimize import minimize
from sympy import symbols, Eq, solve
import math

builder = pymol_jupyter_builder()
builder.start()

In [3]:
# Import cmd 
import xmlrpc.client as xmlrpclib
cmd = xmlrpclib.ServerProxy('http://localhost:9123')
cmd.reinitialize()

## Input: Biochar data

In [4]:
Biochar_Temp=float(input("Enter the temperature of pyrolysis in °C: "))

Enter the temperature of pyrolysis in °C:  500


In [5]:
# Hydroxyl groups, qualitative data from FTIR

if Biochar_Temp == 400:
    v = 0.03
elif Biochar_Temp == 500:
    v = 0.001
elif Biochar_Temp == 600 or Biochar_Temp == 700:
    v = 0
else:
    v = 0  # You may want to specify a default value in case Biochar_Temp doesn't match any of the above conditions

### Ultimate analysis

In [6]:
Carbon = float(input("Enter the value for Carbon: "))
Hydrogen = float(input("Enter the value for Hydrogen: "))
Oxygen = float(input("Enter the value for Oxygen: "))
Nitrogen = float(input("Enter the value for Nitrogen: "))
H_C = float(input("Enter the value for H/C molar ratio: "))
O_C = float(input("Enter the value for O/C molar ratio: "))
BridgheadCarbon=float(input("Enter the value for BridgheadCarbon: "))

Enter the value for Carbon:  0.8141
Enter the value for Hydrogen:  0.0350
Enter the value for Oxygen:  0.1498
Enter the value for Nitrogen:  0.0011
Enter the value for H/C molar ratio:  0.512
Enter the value for O/C molar ratio:  0.137
Enter the value for BridgheadCarbon:  0.488


### Multi-CP 13C NMR quantitaive data

In [8]:
Aromatic = float(input("Enter the value for Caro: "))
Carbonyl = float(input("Enter the value for Carbonyl: "))
Carboxyl = float(input("Enter the value for Carboxyl: "))
Ether = float(input("Enter the value for Ether: "))
Aliphatic = float(input("Enter the value for Aliphatic: "))
Defect = float(input("Enter the value for Defect: "))
C_H_=float(input("Enter the value C-H: "))
C_x_H_=float(input("Enter the value C-O-H: "))

Enter the value for Caro:  0.6733
Enter the value for Carbonyl:  0.0233
Enter the value for Carboxyl:  0.0333
Enter the value for Ether:  0.1233
Enter the value for Aliphatic:  0.0333
Enter the value for Defect:  0.1133
Enter the value C-H:  0.31
Enter the value C-O-H:  0.47


### Aromatic cluster size distribution, obtained from BPCA.ipynb

In [9]:
%store -r result
values=np.round(result[0]).astype(int)
print(values)

[ 0  3  0  0  0 10  0  0  0  0  0  0  2  0  0  0 11  3 62  3  0  0  7  3
  1  2  0  0  0  0  0  2  0]


## Step1: Include the aromatic clusters in PyMOL and create the grid

#### Note: All the molecules included in the BPCA_PAH should be in this folder as mol2 file 

In [10]:
cmd.load("dibenzofuran.mol2", "dibenzofuran")
cmd.load("phenalene.mol2", "phenalene")
cmd.load("phenanthrene.mol2", "phenanthrene")
cmd.load("anthracene.mol2", "anthracene")
cmd.load("tetracene.mol2", "tetracene")
cmd.load("pentacene.mol2", "pentacene")
cmd.load("pyrene.mol2", "pyrene")
cmd.load("chrysene.mol2", "chrysene")
cmd.load("benzo_a_fluorene.mol2", "benzo_a_fluorene")
cmd.load("benzo_b_fluoranthene.mol2", "benzo_b_fluoranthene")
cmd.load("benzo_b_fluorene.mol2", "benzo_b_fluorene")
cmd.load("coronene.mol2", "coronene")
cmd.load("perylene.mol2", "perylene")
cmd.load("benzo_a_pyrene.mol2", "benzo_a_pyrene")
cmd.load("benzo_g_h_i_perylene.mol2", "benzo_g_h_i_perylene")
cmd.load("circumpyrene.mol2", "circumpyrene")
cmd.load("circumcoronene.mol2", "circumcoronene")
cmd.load("circumovalene.mol2", "circumovalene")
cmd.load("pentatriacotaene.mol2", "pentatriacotaene")
cmd.load("circumcircumpyrene.mol2", "circumcircumpyrene")
cmd.load("c84.mol2", "c84")
cmd.load("N5.mol2", "m5")
cmd.load("N8.mol2", "N8")
cmd.load("N9.mol2", "N9")
cmd.load("N10.mol2", "N10")
cmd.load("N11.mol2", "N11")
cmd.load("N12.mol2", "N12")
cmd.load("N13.mol2", "N13")
cmd.load("N14.mol2", "N14")
cmd.load("N15.mol2", "N15")
cmd.load("N16.mol2", "N16")
cmd.load("N17.mol2", "N17")
cmd.load("N18.mol2", "N18")

1

In [11]:
cmd.alias("colour", "color cyan, (name c*); color red, (name o*); color white, (name h*); color blue, (name n*); color yellow, (name s); color brown, (name si)")

In [12]:
cmd.do('colour')
cmd.do('orient')
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')
cmd.bg_color("white")
cmd.set("ray_shadows", "off")

In [13]:
# Define the grid of the molecules
def generate_coordinates():
    coordinates = []
    
    xmin, xmax = 0, 100
    ymin, ymax = -100, 0
    zmin, zmax = 0, 100
    
    x_spacing = 25
    y_spacing = 25
    z_spacing = 25
            
    for x in range(xmin, xmax + 1, x_spacing):
        for y in range(ymin, ymax + 1, y_spacing):
            for z in range(zmin, zmax + 1, z_spacing):
                rotation_angles = [random.uniform(0, 360) for _ in range(3)]
                coordinates.append((x, y, z, rotation_angles))
    
    return coordinates

coordinates = generate_coordinates()
print(len(coordinates))

125


In [14]:
# Count the number of objects
object_list = cmd.get_object_list()
num_objects = len(object_list)
print("PAH types:", num_objects)

PAH types: 33


#### Insert all the PAH in PyMol according to the BPCA distribution

In [15]:
# Copy and translate dibenzofuran
for i in range(1, values[0]+1):
    cmd.copy(f"dibenzofuran{i}", "dibenzofuran")
    cmd.translate(coordinates[i-1+num_objects], f"dibenzofuran{i}")
cmd.delete("dibenzofuran")

# Copy and translate phenalene
for i in range(1, values[1]+1):
    cmd.copy(f"phenalene{i}", "phenalene")
    cmd.translate(coordinates[i-1+num_objects ], f"phenalene{i}")
cmd.delete("phenalene")

# Copy and translate phenanthrene
for i in range(1, values[2]+1):
    cmd.copy(f"phenanthrene{i}", "phenanthrene")
    cmd.translate(coordinates[i-1+num_objects], f"phenanthrene{i}")
cmd.delete("phenanthrene")

# Copy and translate anthracene
for i in range(1, values[3]+1):
    cmd.copy(f"anthracene{i}", "anthracene")
    cmd.translate(coordinates[i-1+num_objects], f"anthracene{i}")
cmd.delete("anthracene")

# Copy and translate tetracene
for i in range(1, values[4]+1):
    cmd.copy(f"tetracene{i}", "tetracene")
    cmd.translate(coordinates[i-1+num_objects], f"tetracene{i}")
cmd.delete("tetracene")

# Copy and translate pentacene
for i in range(1, values[5]+1):
    cmd.copy(f"pentacene{i}", "pentacene")
    cmd.translate(coordinates[i-1+num_objects], f"pentacene{i}")
cmd.delete("pentacene")

# Copy and translate pyrene
for i in range(1, values[6]+1):
    cmd.copy(f"pyrene{i}", "pyrene")
    cmd.translate(coordinates[i-1+num_objects], f"pyrene{i}")
cmd.delete("pyrene")

# Copy and translate chrysene
for i in range(1, values[7]+1):
    cmd.copy(f"chrysene{i}", "chrysene")
    cmd.translate(coordinates[i-1+num_objects], f"chrysene{i}")
cmd.delete("chrysene")

# Copy and translate benzo_a_fluorene
for i in range(1, values[8]+1):
    cmd.copy(f"benzo_a_fluorene{i}", "benzo_a_fluorene")
    cmd.translate(coordinates[i-1+num_objects], f"benzo_a_fluorene{i}")
cmd.delete("benzo_a_fluorene")

# Copy and translate benzo_b_fluoranthene
for i in range(1, values[9]+1):
    cmd.copy(f"benzo_b_fluoranthene{i}", "benzo_b_fluoranthene")
    cmd.translate(coordinates[i-1+num_objects], f"benzo_b_fluoranthene{i}")
cmd.delete("benzo_b_fluoranthene")

# Copy and translate benzo_b_fluorene
for i in range(1, values[10]+1):
    cmd.copy(f"benzo_b_fluorene{i}", "benzo_b_fluorene")
    cmd.translate(coordinates[i-1+num_objects], f"benzo_b_fluorene{i}")
cmd.delete("benzo_b_fluorene")

# Copy and translate coronene
for i in range(1, values[11]+1):
    cmd.copy(f"coronene{i}", "coronene")
    cmd.translate(coordinates[i-1+num_objects], f"coronene{i}")
cmd.delete("coronene")

# Copy and translate perylene
for i in range(1, values[12]+1):
    cmd.copy(f"perylene{i}", "perylene")
    cmd.translate(coordinates[i-1+num_objects], f"perylene{i}")
cmd.delete("perylene")

# Copy and translate benzo_a_pyrene
for i in range(1, values[13]+1):
    cmd.copy(f"benzo_a_pyrene{i}", "benzo_a_pyrene")
    cmd.translate(coordinates[i-1+num_objects], f"benzo_a_pyrene{i}")
cmd.delete("benzo_a_pyrene")

# Copy and translate benzo_g_h_i_perylene
for i in range(1, values[14]+1):
    cmd.copy(f"benzo_g_h_i_perylene{i}", "benzo_g_h_i_perylene")
    cmd.translate(coordinates[i-1+num_objects], f"benzo_g_h_i_perylene{i}")
cmd.delete("benzo_g_h_i_perylene")

# Copy and translate circumpyrene
for i in range(1, values[15]+1):
    cmd.copy(f"circumpyrene{i}", "circumpyrene")
    cmd.translate(coordinates[i-1+num_objects], f"circumpyrene{i}")
cmd.delete("circumpyrene")

# Copy and translate cirumcoronene
for i in range(1, values[16]+1):
    cmd.copy(f"circumcoronene{i}", "circumcoronene")
    cmd.translate(coordinates[i-1+num_objects], f"circumcoronene{i}")
cmd.delete("circumcoronene")

# Copy and translate circumovalene
for i in range(1, values[17]+1):
    cmd.copy(f"circumovalene{i}", "circumovalene")
    cmd.translate(coordinates[i-1+num_objects], f"circumovalene{i}")
cmd.delete("circumovalene")

# Copy and translate pentatriacotaene
for i in range(1, values[18]+1):
    cmd.copy(f"pentatriacotaene{i}", "pentatriacotaene")
    cmd.translate(coordinates[i-1+num_objects], f"pentatriacotaene{i}")
cmd.delete("pentatriacotaene")

# Copy and translate circumcircumpyrene
for i in range(1, values[19]+1):
    cmd.copy(f"circumcircumpyrene{i}", "circumcircumpyrene")
    cmd.translate(coordinates[i-1+num_objects], f"circumcircumpyrene{i}")
cmd.delete("circumcircumpyrene")

# Copy and translate c84
for i in range(1, values[20]+1):
    cmd.copy(f"c84{i}", "c84")
    cmd.translate(coordinates[i-1+num_objects], f"c84{i}")
cmd.delete("c84")

# Copy and translate N5
for i in range(1, values[21]+1):
    cmd.copy(f"m5{i}", "m5")
    cmd.translate(coordinates[i-1+num_objects], f"m5{i}")
cmd.delete("m5")

# Copy and translate N8
for i in range(1, values[22]+1):
    cmd.copy(f"N8{i}", "N8")
    cmd.translate(coordinates[i-1+num_objects], f"N8{i}")
cmd.delete("N8")

# Copy and translate N9
for i in range(1, values[23]+1):
    cmd.copy(f"N9{i}", "N9")
    cmd.translate(coordinates[i-1+num_objects], f"N9{i}")
cmd.delete("N9")

# Copy and translate N10
for i in range(1, values[24]+1):
    cmd.copy(f"N10{i}", "N10")
    cmd.translate(coordinates[i-1+num_objects], f"N10{i}")
cmd.delete("N10")

# Copy and translate N11
for i in range(1, values[25]+1):
    cmd.copy(f"N11{i}", "N11")
    cmd.translate(coordinates[i-1+num_objects], f"N11{i}")
cmd.delete("N11")

# Copy and translate N12
for i in range(1, values[26]+1):
    cmd.copy(f"N12{i}", "N12")
    cmd.translate(coordinates[i-1+num_objects], f"N12{i}")
cmd.delete("N12")

# Copy and translate N13
for i in range(1, values[27]+1):
    cmd.copy(f"N13{i}", "N13")
    cmd.translate(coordinates[i-1+num_objects], f"N13{i}")
cmd.delete("N13")

# Copy and translate N14
for i in range(1, values[28]+1):
    cmd.copy(f"N14{i}", "N14")
    cmd.translate(coordinates[i-1+num_objects], f"N14{i}")
cmd.delete("N14")

# Copy and translate N15
for i in range(1, values[29]+1):
    cmd.copy(f"N15{i}", "N15")
    cmd.translate(coordinates[i-1+num_objects], f"N15{i}")
cmd.delete("N15")

# Copy and translate N16
for i in range(1, values[30]+1):
    cmd.copy(f"N16{i}", "N16")
    cmd.translate(coordinates[i-1+num_objects], f"N16{i}")
cmd.delete("N16")

# Copy and translate N17
for i in range(1, values[31]+1):
    cmd.copy(f"N17{i}", "N17")
    cmd.translate(coordinates[i-1+num_objects], f"N17{i}")
cmd.delete("N17")

# Copy and translate N18
for i in range(1, values[32]+1):
    cmd.copy(f"N18{i}", "N18")
    cmd.translate(coordinates[i-1+num_objects], f"N18{i}")
cmd.delete("N18")

object_list = cmd.get_object_list()
num_objects = len(object_list)
print("PAH clusters:", num_objects)
cmd.reset()


PAH clusters: 109


1

### Check the number of aromatic carbons

In [16]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_aromatic = atom_counts["C"]
h_aromatic = atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6155
Element H: 2118
Element N: 0
Element O: 0
Element S: 0
Element Cl: 0
Element Si: 0


## Step 2: Include cluster with non-hexagonal rings

#### Include a frequency for the ring defects, to have the complete distribution of fragments.

In [17]:
# H/C ratio and cross-linking

if Biochar_Temp == 400:
    w = -0.17   # Factor to adjust how much hydrogen there is to create cross-linking
    y = 1.5     # Factor to increase the amount of defective carbon; the actual value would be corrected later
    f = 10      # Set bounds for the variables (adjust these as needed)
elif Biochar_Temp == 500:
    w = 0.09
    y = 1.25
    f = 10
elif Biochar_Temp == 600:
    w = 0.09
    y = 1.3
    f = 6 
elif Biochar_Temp == 700:
    w = 0.06
    y = 1.45
    f = 6
else:
    w = 0  # You may want to specify a default value in case Biochar_Temp doesn't match any of the above conditions
    y = 1
    f = 10 

In [18]:
from scipy.optimize import minimize

def_list = [107,78, 107, 45, 44, 169, 76, 206, 206, 76, 78]
def_list_H = [0, 116, 162, 74, 74, 224, 112, 274, 0, 0, 0]
TotalC_Aromatic = c_aromatic
TotalH_Aromatic = h_aromatic

value =[]

def constraint(vector, def_list, def_list_H, TotalC_Aromatic, TotalH_Aromatic):
    
    # Total number of defective carbons
    value = [(def_list[i] * vector[i]) / (TotalC_Aromatic + (def_list[i] * vector[i])) for i in range(len(def_list))]
    constraint_value = sum(value)
     
    # Total H/C
    value_H_d = [def_list_H[i] * vector[i] for i in range(len(def_list))]
    value_C_d =  [def_list[i] * vector[i] for i in range(len(def_list))]      
    value_HC = (sum(value_H_d) + TotalH_Aromatic) / (TotalC_Aromatic + sum(value_C_d))
    
    # Total % of carbons
    value_C = (sum(value_C_d)  + TotalC_Aromatic) / (TotalC_Aromatic + TotalH_Aromatic + sum(value_H_d) + sum(value_C_d))
    constraint_value_C = value_C

    # Minimize errors
    error_HC = np.abs(value_HC - (H_C+w))/(H_C+w) # We need more hydrogen to create cross-links    
    error_Def = np.abs(constraint_value - Defect*y)/(Defect*y) # We need more carbon defect to create holes, if you don't want holes put y=1
    
    Total_error = error_Def + error_HC

    return Total_error*100

# Initial guess for the optimization
initial_vector = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

bounds = [(0, f) for i in range(len(def_list))]

# Optimize the objective function (set to zero) subject to constraints
result = minimize(lambda x: 0, initial_vector, method='slsqp', constraints={'type': 'eq', 'fun': constraint, 'args': (def_list, def_list_H, TotalC_Aromatic, TotalH_Aromatic)}, bounds=bounds, tol=1e-5)

# Extract the optimized vector
vector = result.x.round().astype(int)

# Print the results
print("Value:", constraint(vector, def_list, def_list_H, TotalC_Aromatic, TotalH_Aromatic))
print("Optimized Vector:", vector)

Value: 17.722106725122487
Optimized Vector: [ 0  0  4  1 10  0  0  0  0  0  0]


In [19]:
# New vector with defective structures included
values = np.append(values, vector).astype(int)
print(values)

[ 0  3  0  0  0 10  0  0  0  0  0  0  2  0  0  0 11  3 62  3  0  0  7  3
  1  2  0  0  0  0  0  2  0  0  0  4  1 10  0  0  0  0  0  0]


In [20]:
# Upload defects
cmd.load("defect1.mol2", "Cdefect")
cmd.load("defect2.mol2", "defect2")
cmd.load("defect3.mol2", "defect3")
cmd.load("defect4.mol2", "defect4")
cmd.load("defect5.mol2", "defect5")
cmd.load("defect6.mol2", "defect6")
cmd.load("defect7.mol2", "defect7")
cmd.load("defect8.mol2", "defect8")
cmd.load("defect9.mol2", "Cdefect9")
cmd.load("defect10.mol2", "Cdefect10")
cmd.load("defect11.mol2", "Cdefect11")

1

In [21]:
# Copy and translate defects
for i in range(1, values[33]+1):
    cmd.copy(f"Cdefect{i}", "Cdefect")
    cmd.translate(coordinates[i-1+num_objects+1], f"Cdefect{i}")
cmd.delete("Cdefect")
for i in range(1, values[35]+1):
    cmd.copy(f"defect3{i}", "defect3")
    cmd.translate(coordinates[i-1+num_objects+2], f"defect3{i}")
cmd.delete("defect3")
for i in range(1, values[34]+1):
    cmd.copy(f"defect2{i}", "defect2")
    cmd.translate(coordinates[i-1+num_objects+3], f"defect2{i}")
cmd.delete("defect2")
for i in range(1, values[36]+1):   
    cmd.copy(f"defect4{i}", "defect4")  
    cmd.translate(coordinates[i-1+num_objects+4], f"defect4{i}")
cmd.delete("defect4")
for i in range(1, values[37]+1):    
    cmd.copy(f"defect5{i}", "defect5")
    cmd.translate(coordinates[i-1+num_objects+5], f"defect5{i}")
cmd.delete("defect5")
for i in range(1, values[38]+1):    
    cmd.copy(f"defect6{i}", "defect6")
    cmd.translate(coordinates[i-1+num_objects+6], f"defect6{i}")
cmd.delete("defect6")
for i in range(1, values[39]+1):    
    cmd.copy(f"defect7{i}", "defect7")
    cmd.translate(coordinates[i-1+num_objects+7], f"defect7{i}")
cmd.delete("defect7")
for i in range(1, values[40]+1):    
    cmd.copy(f"defect8{i}", "defect8")
    cmd.translate(coordinates[i-1+num_objects+8], f"defect8{i}")
cmd.delete("defect8")
for i in range(1, values[41]+1):    
    cmd.copy(f"Cdefect9{i}", "Cdefect9")
    cmd.translate(coordinates[i-1+num_objects+9], f"Cdefect9{i}")
cmd.delete("Cdefect9")
for i in range(1, values[42]+1):    
    cmd.copy(f"Cdefect10{i}", "Cdefect10")
    cmd.translate(coordinates[i-1+num_objects+10], f"Cdefect10{i}")
cmd.delete("Cdefect10")
for i in range(1, values[43]+1):    
    cmd.copy(f"Cdefect11{i}", "Cdefect11")
    cmd.translate(coordinates[i-1+num_objects+11], f"Cdefect11{i}")
cmd.delete("Cdefect11")

# Count the number of objects
object_list = cmd.get_object_list()
num_objects = len(object_list)

# Print the count
print("Number of objects:", num_objects)
cmd.reset()

Number of objects: 124


1

In [22]:
cmd.do('colour')
cmd.do('comp')
cmd.do('atomdata')
cmd.do('orient')
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')

### Organize the structures without overlapping them; this will allow you to visualize better the changes made

In [23]:
# Define the range of rotation angles in degrees
min_angle = 1
max_angle = 359.0

# Define the number of divisions in each dimension of the grid
grid_divisions = 15

# Track the positions and angles of molecules
positions = []
angles = []
xmin, xmax = 0, 100
ymin, ymax = -100, 0
zmin, zmax = 0, 100

# Shuffle the object_list to randomize the order
random.shuffle(object_list)

# Calculate the grid size in each dimension
grid_size_x = (xmax - xmin) / grid_divisions
grid_size_y = (ymax - ymin) / grid_divisions
grid_size_z = (zmax - zmin) / grid_divisions

for name in object_list:
    unique_position = False
    while not unique_position:
        # Calculate the grid indices for the position
        grid_index_x = random.randint(0, grid_divisions - 1)
        grid_index_y = random.randint(0, grid_divisions - 1)
        grid_index_z = random.randint(0, grid_divisions - 1)

        # Calculate the position within the grid cell
        x = xmin + (grid_index_x + random.uniform(0, 1)) * grid_size_x
        y = ymin + (grid_index_y + random.uniform(0, 1)) * grid_size_y
        z = zmin + (grid_index_z + random.uniform(0, 1)) * grid_size_z

        new_position = (x, y, z)

        # Check if the new position conflicts with any existing positions or angles
        if all(all(abs(new_position[i] - pos[i]) > 1e-6 for i in range(3)) for pos in positions) and new_position not in positions:
            unique_position = True
            positions.append(new_position)

            # Calculate a new random angle
            rotation_angle = random.randint(min_angle, max_angle)
            new_angle = (rotation_angle, name)

            angles.append(new_angle)

            # Perform rotation on the object
            for axis in ['x', 'y', 'z']:
                rotation_command = "rotate {0}, {1}, {2}".format(axis, rotation_angle, name)
                cmd.do(rotation_command)

# Update the display
cmd.refresh()

In [24]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total= atom_counts["C"]
h_total= atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7068
Element H: 3580
Element N: 0
Element O: 0
Element S: 0
Element Cl: 0
Element Si: 0


In [25]:
c_defect=c_total-c_aromatic
print("Defect carbon:", c_defect)

Defect carbon: 913


## Step 3: Create the distribution to add functional groups


### Add additional groups as needed

In [26]:
# Molecule type and functional groups to be included, information obtained from FTIR, CNMR, and XPS 
# The user defines these:             
Furan=0           # Change: interior C 
Pyrrolic=0        # Change: interior C
Fluorene=0        # Change: interior 
Thiophene=0       # Change: interior

# To reduce the error of not having enough information for oxygen, 
# it is necessary to sacrifice the precision of functional groups 
# that contain oxygen, such as ether, carbonyl, and carboxylic groups.

if Biochar_Temp == 400:
    z = -0.05   # A factor to fix oxygen based on ether, carbonyl, and carboxyl groups
    u = 0       # How much of the aliphatic is methyl
    b = 0       # How much of the aliphatic is CH2CH3
    p = 1       # How much of the aliphatic is (CH2)-(CH2)-CH3
    h = 0.05    # A factor to fix oxygen based on defects
    q = -0.05   # A factor to fix oxygen based on aliphatic groups
elif Biochar_Temp == 500:
    z = 0.05
    u = 1/3
    b = 2/3
    p = 0
    h = 0
    q = 0.05
elif Biochar_Temp == 600:
    z = 0.05
    u = 0
    b = 0
    p = 0
    h = 0
    q = 0.09
elif Biochar_Temp == 700:
    z = 0.1
    u = 0
    b = 0
    p = 0
    h = 0
    q = 0.05
else:
    z = 0  # Specify a default value in case Biochar_Temp doesn't match any of the above conditions
    u = 0
    b = 0
    p = 0

### Fix carbon, nitrogen, oxygen, and hydrogen

In [27]:
# Variables
c, carb, carbox, eth, alip, defe, c_nonar,aro_ring, methyl, ali_chain, hydro, aro, nitro_groups, aniline, pyridin, quaternaryN, MW, oxy, hydroxyl,ali_chain_2 = symbols('c carb carbox eth aro_ring c_nonar alip defe methyl ali_chain hydro aro nitro_groups aniline pyridin quaternaryN MW oxy hydroxyl ali_chain_2')
# Equations
equations = [
    
    # Carbon
    Eq(c_nonar, defe+methyl+ali_chain*2+ali_chain_2*3+carb+carbox),# Non-aromatic carbons in the model
    Eq(defe,Defect*c*(1+q)),
    Eq(c,(c_nonar+aro_ring)),
    Eq(aro,(aro_ring-aniline-hydroxyl-carbox-carb-methyl-ali_chain-eth)), # Actual number of aromatic carbons in the model
    Eq(aro_ring,(c_aromatic-eth-pyridin-quaternaryN)),            # Aromatic carbons removed
    Eq(carb, Carbonyl*c*(0.95-z)),                                # Change: aromatic H
    Eq(carbox, Carboxyl*c*(0.95-z)),                              # Change: aromatic H
    Eq(eth,0.5*Ether*c*(0.95-z)),                                 # Change: aromatic C-H (In NMR ether groups count two carbons per oxygen)
    Eq(alip, Aliphatic*c*(1-h)),            
    Eq(methyl,(alip*u)/1),                                        # Change: aromatic H
    Eq(ali_chain,(alip*b)/2),                                     # Change: aromatic H
    Eq(ali_chain_2,(alip*p)/3),                                   # Change: aromatic H
    
    # Nitrogen
    Eq(nitro_groups,(MW*Nitrogen)/14.0067),
    Eq(aniline,nitro_groups*2/4),                                  # Change: aromatic H
    Eq(pyridin,nitro_groups*1/4),                                  # Change: aromatic C-H
    Eq(quaternaryN,nitro_groups*1/4),                              # Change: interior C
    
    # Hydrogen
    Eq(hydro,(h_total+methyl*2+ali_chain*4+ali_chain_2*6+nitro_groups*3/4-carb-eth)),
    
    # Oxygen
    Eq(hydroxyl,((MW*Oxygen)/15.999)*v),                           # Change: aromatic H
    Eq(oxy,hydroxyl+carbox*2+carb+eth),
    
    # Molecular weight
    Eq(MW,(c*12.011+hydro*1.00784+nitro_groups*14.0067+oxy*15.999))
     
]

# Solve the equations
solution = solve(equations, (c, carb, carbox, eth, alip, c_nonar ,defe, aro_ring,methyl, ali_chain, hydro, aro, nitro_groups, aniline,pyridin, quaternaryN, MW, oxy, hydroxyl, ali_chain_2))

# Print the solutions
print("aro =", round(solution[aro]))
print("aro_ring =", round(solution[aro_ring]))
print("carb =", round(solution[carb]))
print("carbox =", round(solution[carbox]))
print("eth =", round(solution[eth]))
print("alip =", math.ceil(solution[alip]))
print("defe =", round(solution[defe]))
print("c_nonar =", round(solution[c_nonar]))
print("methyl =", round(solution[methyl]))
print("ali_chain =", math.ceil(solution[ali_chain]))
print("ali_chain_2 =", math.ceil(solution[ali_chain_2]))
print("aniline =", round(solution[aniline]))
print("pyridin =", round(solution[pyridin]))
print("quaternaryN =", math.ceil(solution[quaternaryN]))
print("Hydroxyl =", round(solution[hydroxyl]))
print("MW =", round(solution[MW]))
print("Oxygen =", round(solution[carb])+round(solution[carbox])*2+ round(solution[eth]) +round(solution[hydroxyl]))
print("Nitrogen =", math.ceil(solution[nitro_groups]))
print("Carbon =", round(solution[c]))
print("Hydrogen =", round(solution[hydro]))


aro = 4817
aro_ring = 5750
carb = 151
carbox = 216
eth = 400
alip = 241
defe = 859
c_nonar = 1467
methyl = 80
ali_chain = 81
ali_chain_2 = 0
aniline = 4
pyridin = 2
quaternaryN = 3
Hydroxyl = 1
MW = 106106
Oxygen = 984
Nitrogen = 9
Carbon = 7217
Hydrogen = 3515


### Create "holes" within the structure

In [28]:
Internal_C_to_remove = round(c_defect - solution[defe])
print("Carbon to remove:", round(Internal_C_to_remove))

Carbon to remove: 54


In [29]:
# Adjust the excess of carbon
if Internal_C_to_remove < 0:
    Internal_C_to_remove = 0
if Internal_C_to_remove >= (num_objects*0.9):
    Internal_C_to_remove = round(Internal_C_to_remove*0.9)

if Biochar_Temp == 400:
    Internal_C_to_remove_Aro=0
    Internal_C_to_remove_Def=Internal_C_to_remove
else:
    Internal_C_to_remove_Aro=Internal_C_to_remove*0.9
    Internal_C_to_remove_Def=Internal_C_to_remove*0.1

print("Carbon to remove in defective structures:", round(Internal_C_to_remove_Def))
print("Carbon to remove in aromatic structures:", round(Internal_C_to_remove_Aro))

Carbon to remove in defective structures: 5
Carbon to remove in aromatic structures: 49


### Define cross-links by fixing H/C ratio

In [30]:
def calculate_Err_H_C(i):
    mw = (round(solution[c])-Internal_C_to_remove)* 12.011 + i * 1.00784 + (round(solution[aniline]) + round(solution[pyridin]) + round(solution[quaternaryN]))* 14.0067 + (round(solution[carb])+round(solution[carbox])*2+ round(solution[eth]) +round(solution[hydroxyl])) * 15.999
    H_C_mod = i/round(solution[c])
    Err_H_C = (abs(H_C_mod - H_C) / H_C) * 100
    return Err_H_C

# Iterate over possible values of i and find the minimum error for H/C ratio
min_error = np.inf
best_i = None
for i in range(5000):  # Adjust the range as needed based on how many carbons you have in the model
    error = calculate_Err_H_C(i)
    if error < min_error:
        min_error = error
        best_i = i

print("Hydrogen needed:", best_i)
print("Error:", round(min_error,3))

Hydrogen needed: 3695
Error: 0.003


In [31]:
Hydrogen_to_remove=round(solution[hydro])-best_i
if Hydrogen_to_remove < 0:
      Hydrogen_to_remove = 0
print("Hydrogen to remove:", Hydrogen_to_remove)
crosslinks=round(Hydrogen_to_remove/2)
print("crosslinks:", crosslinks)
newhydro=round(solution[hydro]-Hydrogen_to_remove)
newMW = (round(solution[c])-Internal_C_to_remove)* 12.011 + newhydro * 1.00784 + (round(solution[aniline]) + round(solution[pyridin]) + round(solution[quaternaryN]))* 14.0067 + (round(solution[carb])+round(solution[carbox])*2+ round(solution[eth]) +round(solution[hydroxyl])) * 15.999

Hydrogen to remove: 0
crosslinks: 0


### See if you have an excess of oxygen


In [32]:
def calculate_Err_O(j):
    mw = (solution[c]-Internal_C_to_remove)* 12.011 + newhydro * 1.00784 + ((solution[aniline] +solution[pyridin] +solution[quaternaryN])* 14.0067) + j * 15.999
    OxygenMoles = (j * 15.999) / mw
    Err_O = (abs(OxygenMoles - Oxygen) / Oxygen)*100
    return Err_O

# Iterate over possible values of i and find the minimum error for O/C ratio
min_error = np.inf
best_j = None
for j in range(5000):  # Adjust the range as needed based on how many carbons you have in the simulation
    error = calculate_Err_O(j)
    if error < min_error:
        min_error = error
        best_j = j

print("Oxygen needed:", best_j)
print("Error:", round(min_error,3))

Oxygen needed: 988
Error: 0.020


### Before modifying check: CNMR data

In [33]:
# Aromatic Carbon
Err_C=(abs(((solution[aro]-Internal_C_to_remove_Aro)/(solution[c]-Internal_C_to_remove))-Aromatic)/Aromatic)*100
print("Aromatic Carbon:",round(((solution[aro]-Internal_C_to_remove_Aro)/(solution[c]-Internal_C_to_remove)),3))
print("Error:",round(Err_C,3))

Aromatic Carbon: 0.666
Error: 1.128


In [34]:
# Carbonyl Carbon
Err_Car=(abs((solution[carb]/(solution[c]-Internal_C_to_remove))-Carbonyl)/Carbonyl)*100
print("Carbonyl Carbon",round(solution[carb]/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_Car,3))

Carbonyl Carbon 0.021
Error: 9.322


In [35]:
# Carboxyl Carbon
Err_Carb=(abs((solution[carbox]/(solution[c]-Internal_C_to_remove))-Carboxyl)/Carboxyl)*100
print("Carboxyl Carbon",round(solution[carbox]/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_Carb,3))

Carboxyl Carbon 0.030
Error: 9.322


In [36]:
# Ether Carbon
Err_ethh=(abs(((solution[eth]*2)/(solution[c]-Internal_C_to_remove))-Ether)/Ether)*100
print("Ether Carbon",round((solution[eth]*2)/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_ethh,3))

Ether Carbon 0.112
Error: 9.322


In [37]:
# Aliphatic Carbon
Err_ali=(abs((solution[alip]/(solution[c]-Internal_C_to_remove))-Aliphatic)/Aliphatic)*100
print("Aliphatic Carbon",round(solution[alip]/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_ali,3))

Aliphatic Carbon 0.034
Error: 0.754


In [38]:
# Defect Carbon
Err_defe=(abs((c_defect-Internal_C_to_remove)/(solution[c]-Internal_C_to_remove)-Defect))/Defect*100
print("Defect Carbon",round((c_defect-Internal_C_to_remove)/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_defe,3))

Defect Carbon 0.120
Error: 5.846


### Before modifying check: Elemental composition 

In [39]:
# Carbon
Elem_c=((solution[c]-Internal_C_to_remove)*12.011)/newMW
print("Carbon:",round(Elem_c,4))
Err_elemc=(abs(Elem_c-Carbon)/Carbon)*100
print("Error:",round(Err_elemc,3))

Carbon: 0.8160
Error: 0.234


In [40]:
# Nitrogen
Elem_n=(solution[nitro_groups]*14.0067)/newMW
print("Nitrogen:",round(Elem_n,4))
Err_elemn=(abs(Elem_n-Nitrogen)/Nitrogen)*100
print("Error:",round(Err_elemn,3))

Nitrogen: 0.0011
Error: 0.639


In [41]:
# Oxygen
Elem_o=((round(solution[carb])+round(solution[carbox])*2+ round(solution[eth]) +round(solution[hydroxyl]))*15.999)/newMW
print("Oxygen:",round(Elem_o,3))
Err_elemo=(abs(Elem_o-Oxygen)/Oxygen)*100
print("Error:",round(Err_elemo,3))

Oxygen: 0.149
Error: 0.321


In [42]:
# Hydrogen
Elem_h=(newhydro*1.00784)/newMW
print("Hydrogen:",round(Elem_h,3))
Err_elemh=(abs(Elem_h-Hydrogen)/Hydrogen)*100
print("Error:",round(Err_elemh,3))

Hydrogen: 0.034
Error: 3.999


## Step 4: Modify structures and include functional groups

In [43]:
object_masses= builder.Get_Object_Masses()
object_names = list(object_masses.keys())
while not object_names:
    object_masses = builder.Get_Object_Masses()
    object_names = list(object_masses.keys())

### Include Ether Groups

In [44]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_totali= atom_counts["C"]
h_totali= atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7068
Element H: 3580
Element N: 0
Element O: 0
Element S: 0
Element Cl: 0
Element Si: 0


In [45]:
#List of object names
object_names = list(object_masses.keys())

# List to store the modified object 
modified_ether = []

if Biochar_Temp == 400 or Biochar_Temp == 500:
    k1 = 0.85
else:
    k1 = 1

# Number of molecules to modify
num_molecules = round(solution[eth]*k1)

excluded_prefixes = ["circumovalene","circumcoronene","circumpyrene","pentatriacotaene", "circumcircumpyrene","C84","N"]
filtered_object_names = [name for name in object_names if any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the filtered list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        c_aro_types = builder.examine_main(mol_to_edit)

        if len(c_aro_types['O_2n']) <= round(O_C*55):
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("carbon_not_bonded_to_oxygen", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("carbon_not_bonded_to_oxygen")
        
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value=random.choice(c_aro_types['C_2n'])
        if value in c_choices:
            break

    # Attach Ether group to the selected carbon atom
    object_ether = cmd.alter(f'{mol_to_edit} and index {value}', 'elem="O"')
    object_ether = cmd.alter(f'{mol_to_edit} and index {value}', 'text_type="O.3"')
    object_ether = cmd.alter(f'{mol_to_edit} and index {value}', 'name="O"')

    # Store the modified ether
    modified_ether.append(object_ether)
    h_types = builder.examine_h(mol_to_edit)
    h_choice = random.choice(h_types['Aliphatic_O_inRing'])
    object_ether = cmd.remove(f'{mol_to_edit} and index {h_choice}')
    modified_ether.append(object_ether)


In [46]:
#List of object names
object_names = list(object_masses.keys())

# List to store the modified object 
modified_ether = []

if Biochar_Temp == 400 or Biochar_Temp == 500:
    k = 0.15
else :
    k = 0

# Number of molecules to modify
num_molecules = math.ceil(solution[eth]*k)

excluded_prefixes = ["circumovalene","circumcoronene","circumpyrene","pentatriacotaene", "circumcircumpyrene","C84","defect","Cdefect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]


# Select a specific number of molecules from the filtered list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        c_aro_types = builder.examine_main(mol_to_edit)

        if len(c_aro_types['O_2n']) <= round(O_C*20):
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("carbon_not_bonded_to_oxygen", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("carbon_not_bonded_to_oxygen")
        
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value=random.choice(c_aro_types['C_2n'])
        if value in c_choices:
            break

    # Attach Ether group to the selected carbon atom
    object_ether = cmd.alter(f'{mol_to_edit} and index {value}', 'elem="O"')
    object_ether = cmd.alter(f'{mol_to_edit} and index {value}', 'text_type="O.3"')
    object_ether = cmd.alter(f'{mol_to_edit} and index {value}', 'name="O"')

    # Store the modified ether
    modified_ether.append(object_ether)
    h_types = builder.examine_h(mol_to_edit)
    h_choice = random.choice(h_types['Aliphatic_O_inRing'])
    object_ether = cmd.remove(f'{mol_to_edit} and index {h_choice}')
    modified_ether.append(object_ether)


In [47]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total= atom_counts["C"]
h_total= atom_counts["H"]
o_total= atom_counts["O"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6667
Element H: 3179
Element N: 0
Element O: 401
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero


In [48]:
cmd.select("carbon_bonded_to_oxygen", "elem O and neighbor elem O")

0

In [49]:
check_ether_o=o_total-math.ceil(solution[eth]*k)-round(solution[eth]*k1)
print(check_ether_o)
check_ether_h=h_totali-h_total-math.ceil(solution[eth]*k)-round(solution[eth]*k1)
print(check_ether_h)
check_ether_c=c_totali-c_total-math.ceil(solution[eth]*k)-round(solution[eth]*k1)
print(check_ether_c)

0
0
0


### Create holes: Aromatic structures

In [50]:
# List of object names
object_names = list(object_masses.keys())

# List to store the modified object 
modified_hole = []

# Number of molecules to modify
num_molecules = round(Internal_C_to_remove_Aro)

excluded_prefixes = ["circumcoronene","circumpyrene","pentatriacotaene", "circumcircumpyrene","C84","N"]
filtered_object_names = [name for name in object_names if any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the filtered list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    
    c_types = builder.examine_main(mol_to_edit)
    c_choice = random.choice(c_types['C_3n'])

    # Remove the internal carbon atom to create a hole in the structure
    object_hole = cmd.remove(f'{mol_to_edit} and index {c_choice}')

    # Store the modified molecules  
    modified_hole.append(object_hole)

In [51]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total2= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6618
Element H: 3179
Element N: 0
Element O: 401
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero

In [52]:
check_c=c_total-(c_total2+round(Internal_C_to_remove_Aro))
print(check_c)

0


### Create holes: Ring defects structures

In [53]:
import random

# List of object names 
object_names = list(object_masses.keys())

# List to store the modified object 
modified_hole = []

# Number of molecules to modify
num_molecules = round(Internal_C_to_remove_Def)

excluded_prefixes = ["Cdefect","defect"]
filtered_object_names = [name for name in object_names if any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Function to select a molecule excluding the last selected one
def select_unique_molecule(last_selected, available_molecules):
    available_molecules = [mol for mol in available_molecules if mol != last_selected]
    if not available_molecules:
        return None
    return random.choice(available_molecules)

last_selected_molecule = None

for _ in range(num_molecules):
    mol_to_edit = select_unique_molecule(last_selected_molecule, filtered_object_names)
    
    if mol_to_edit is None:
        
        break
    
    c_types = builder.examine_main(mol_to_edit)
    c_choice = random.choice(c_types['C_3n'])

    # Remove the internal carbon atom to create a hole in the structure
    object_hole = cmd.remove(f'{mol_to_edit} and index {c_choice}')

    # Store the modified molecules
    modified_hole.append(object_hole)
    
    # Update last_selected_molecule
    last_selected_molecule = mol_to_edit


In [54]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total2_d= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6613
Element H: 3179
Element N: 0
Element O: 401
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero

In [55]:
check_c=c_total2-(c_total2_d+round(Internal_C_to_remove_Def))
print(check_c)

0


###  Include Pyridinic Group 

In [56]:
#List of object names
object_names = list(object_masses.keys())

# List to store the modified object 
modified_pyridin = []

# Number of molecules to modify
num_molecules = round(solution[pyridin])

# Exclude object names that start with specific prefixes
excluded_prefixes = ["Cdefect","defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the filtered list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)


# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        n_types = builder.examine_main(mol_to_edit)

        if len(n_types['N_2n'])<1:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("nitrogen_not_bonded_to_nitrogen", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("nitrogen_not_bonded_to_nitrogen")
        
    n_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value_n=random.choice(n_types['C_2n'])
        if value_n in n_choices:
            break

    # Attach Ether group to the selected carbon atom
    object_pyridin = cmd.alter(f'{mol_to_edit} and index {value_n}', 'elem="N"')
    object_pyridin = cmd.alter(f'{mol_to_edit} and index {value_n}', 'text_type="N.2"')
    object_pyridin = cmd.alter(f'{mol_to_edit} and index {value_n}', 'name="N"')

    # Store the modified pyrridin
    modified_pyridin.append(object_pyridin)
    h_types = builder.examine_h(mol_to_edit)
    h_choice = random.choice(h_types['Aliphatic_N_inRing'])
    object_pyridin = cmd.remove(f'{mol_to_edit} and index {h_choice}')

    # Store the modified pyrridin
    modified_pyridin.append(object_pyridin)

In [57]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
n_total= atom_counts["N"]
h_totali= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6611
Element H: 3177
Element N: 2
Element O: 401
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero


In [58]:
check_n=n_total-round(solution[pyridin])
print(check_n)

0


In [59]:
cmd.select("nitrogen_bonded_to_oxygen_nitrogen", "elem N and neighbor elem O")

0

In [60]:
cmd.select("nitrogen_bonded_nitrogen", "elem N and neighbor elem N")

0

### Include Hydroxyl Groups

In [61]:
# List of object names 
object_names = list(object_masses.keys())

# List to store the modified object 
modified_hydroxyls = []

# Number of molecules to modify
num_molecules = round(solution[hydroxyl])

# Exclude object names that start with specific prefixes
excluded_prefixes = ["Cdefect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    
    # Get available hydrogen types
    while True:
        
        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)

        if len(o_types['O_2n'])<3 and len(o_types['O_1n'])< 3:
            break  # Condition met, proceed to modify the molecule
        
        mol_to_edit = random.choice(filtered_object_names)
    
    h_types = builder.examine_h(mol_to_edit)
    
    # Check if the molecule has 'Aliphatic_C_inRing' oxygen type
    if len(h_types['Aliphatic_C_inRing']) == 0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else: 
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])

        # Attach hydroxyl group
        object_hydroxyl = builder.Attach_OH(mol_to_edit, h_choice)
    
    # Store the modified hydroxyl
    modified_hydroxyls.append(object_hydroxyl)

In [62]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
o_total= atom_counts["O"]
c_total= atom_counts["C"]
h_total= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6611
Element H: 3177
Element N: 2
Element O: 402
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [63]:
check_ox=o_total-math.ceil(solution[eth]*k)-round(solution[eth]*k1)-round(solution[hydroxyl])
print(check_ox)
check_hy=h_totali-h_total
print(check_hy)

0
0


### Include Carbonyl Groups

In [64]:
# List of object names
object_names = list(object_masses.keys())

# List to store the modified object
modified_carbonyl = []

# Number of molecules to modify
num_molecules = round(solution[carb])


# Exclude object names that start with specific prefixes
excluded_prefixes = ["benzo_a_fluorene", "benzo_b_fluorene","phenalene", "phenanthrene","anthracene","pyrene","benzo_a_pyrene","perylene","chrysene","benzo_b_fluoranthene","Cdefect","N5"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the filtered list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:

        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")
            
        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]
        
        if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*40) and len(o_types['O_1n'])<round(O_C*40):
            break  # Condition met, proceed to modify the molecule
            
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit) 

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach carbonyl group
    object_carbonyl = builder.Attach_Carbonyl(mol_to_edit, h_choice)

    # Store the modified carbonyl
    modified_carbonyl.append(object_carbonyl)

In [65]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
o_total_new= atom_counts["O"]
c_total_new= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6762
Element H: 3177
Element N: 2
Element O: 553
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [66]:
check_c=c_total_new-(c_total+round(solution[carb]))
print(check_c)
check_o=o_total_new-(o_total+round(solution[carb]))
print(check_o)

0
0


### Include Carboxyl Groups

In [67]:
# List of object names
object_names = list(object_masses.keys())

# List to store the modified object 
modified_carboxyl = []

# Number of molecules to modify
num_molecules = round(solution[carbox])

# Exclude object names that start with specific prefixes
excluded_prefixes = ["Cdefect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
            
        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]
        
        if Biochar_Temp==700:
            if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*80) and len(o_types['O_1n'])<round(O_C*80):
                break  # Condition met, proceed to modify the molecule
        else:
             if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*55) and len(o_types['O_1n'])<round(O_C*55):
                break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing']) == 0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach carboxyl group
    object_carboxyl = builder.Attach_Carboxyl(mol_to_edit, h_choice)

    # Store the modified carboxyl
    modified_carboxyl.append(object_carboxyl)

In [68]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
o_total_new2= atom_counts["O"]
c_total_new2= atom_counts["C"]
h_total_new2= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6978
Element H: 3177
Element N: 2
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [69]:
check_c=c_total_new2-(c_total_new+round(solution[carbox]))
print(check_c)
check_o=o_total_new2-(o_total_new+round(solution[carbox])*2)
print(check_o)

0
0


### Include Methyl Groups

In [70]:
# List of object names
object_names = list(object_masses.keys())

# List to store the modified object
modified_methyl = []

# Number of molecules to modify
num_molecules = round(solution[methyl])

# Exclude object names that start with specific prefixes
excluded_prefixes = ["Cdefect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:  

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>4:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break
            
    # Attach Methyl group 
    object_methyl = builder.Attach_CH3(mol_to_edit, h_choice)

    # Store the modified methyl
    modified_methyl.append(object_methyl)

In [71]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total_new= atom_counts["C"]
h_total_new= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7058
Element H: 3337
Element N: 2
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [72]:
check_c=c_total_new-(c_total_new2+round(solution[methyl]))
print(check_c)
check_hy=h_total_new-(h_total_new2+round(solution[methyl])*2)
print(check_hy)

0
0


### Include CH2CH3 Groups

In [73]:
# List of object names 
object_names = list(object_masses.keys())

# List to store the modified object
modified_ali_chain = []

# Number of molecules to modify
num_molecules = round(solution[ali_chain])

# Exclude object names that start with specific prefixes
excluded_prefixes = ["CdefectC"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>8:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Aliphatic chain group           
    object_ali_chain = builder.Attach_CH2CH3(mol_to_edit, h_choice)

    # Store the modified ali chain
    modified_ali_chain.append(object_ali_chain)

In [74]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total= atom_counts["C"]
h_total= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7218
Element H: 3657
Element N: 2
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [75]:
check_c=c_total-(c_total_new+round(solution[ali_chain])*2)
print(check_c)
check_hy=h_total-(h_total_new+round(solution[ali_chain])*4)
print(check_hy)

0
0


### Include (CH2)2-CH3 Groups for Biochar at 400°C

In [76]:
# List of object names 
object_names = list(object_masses.keys())

# List to store the modified object
modified_ali_chain_2 = []

# Number of molecules to modify
num_molecules = round(solution[ali_chain_2]/2)

# Exclude object names that start with specific prefixes
excluded_prefixes = ["Cdefect","defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>10:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    if len(h_types['Aliphatic_C_inRing'])==0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else:
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])
     
    # Attach Aliphatic chain group           
    object_ali_chain2 = builder.Attach_C2H4CH3(mol_to_edit, h_choice)

    # Store the modified ali chain
    modified_ali_chain_2.append(object_ali_chain2)

In [77]:
# List of object names 
object_names = list(object_masses.keys())

# List to store the modified object
modified_ali_chain_2 = []

# Number of molecules to modify
num_molecules = round(solution[ali_chain_2]/2)

# Exclude object names that start with specific prefixes
excluded_prefixes = ["Cdefect","defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>12:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    if len(h_types['Aliphatic_C_inRing'])==0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else:
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])
     
    # Attach Aliphatic chain group           
    object_ali_chain2 = builder.Attach_C2H4CH3(mol_to_edit, h_choice)

    # Store the modified ali chain
    modified_ali_chain_2.append(object_ali_chain2)

In [78]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total_2= atom_counts["C"]
h_total_2= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7218
Element H: 3657
Element N: 2
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [79]:
check_c=c_total_2-(c_total+round(solution[ali_chain_2])*3)
print(check_c)
check_hy=h_total_2-(h_total+round(solution[ali_chain_2])*6)
print(check_hy)

0
0


### Include NH3 Groups

In [80]:
# List of object names
object_names = list(object_masses.keys())

# List to store the modified object
modified_Aniline = []

# Number of molecules to modify
num_molecules = round(solution[aniline])

# Exclude object names that start with specific prefixes
excluded_prefixes = ["Cdefect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:           
    while True:

        # Get available carbon types
        n_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(n_types['N_2n'])==0 and len(h_choices)>7 and len(n_types['N_3n']) == 0 and len(n_types['N_1n']) == 0:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach aniline group
    object_Aniline = builder.Attach_Aniline(mol_to_edit, h_choice)

    # Store the modified aniline
    modified_Aniline.append(object_Aniline)

In [81]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
n_total= atom_counts["N"]
h_total_new= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7218
Element H: 3661
Element N: 6
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [82]:
check_n=n_total-(round(solution[aniline])+ round(solution[pyridin]))
print(check_n)
check_h=h_total_new-(h_total_2+round(solution[aniline]))
print(check_h)

0
0


### Include Quaternary Nitrogen

In [83]:
#List of object names 
object_names = list(object_masses.keys())

# List to store the modified object 
modified_quater = []

# Number of molecules to modify
num_molecules = math.ceil(solution[quaternaryN])

excluded_prefixes = ["circumcoronene","circumpyrene","pentatriacotaene", "circumcircumpyrene","C84"]
filtered_object_names = [name for name in object_names if any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the filtered list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

    # Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    
    while True:
        # Get available carbon types
        n_types = builder.examine_main(mol_to_edit)

        if len(n_types['N_2n']) == 0 and len(n_types['N_3n']) == 0 and len(n_types['N_1n']) == 0:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
            
    cmd.select("nitrogen_not_bonded_to_nitrogen", f"({mol_to_edit} and elem C and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("nitrogen_not_bonded_to_nitrogen")
        
    n_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value_n=random.choice(n_types['C_3n'])
        if value_n in n_choices:
            break
    
    #value_n=random.choice(n_types['C_3n'])
    
    # Attach Ether group to the selected carbon atom
    object_quater = cmd.alter(f'{mol_to_edit} and index {value_n}', 'elem="N"')
    object_quater = cmd.alter(f'{mol_to_edit} and index {value_n}', 'text_type="NN"')
    object_quater = cmd.alter(f'{mol_to_edit} and index {value_n}', 'name="N"')
    
    modified_quater.append(object_quater)
    h_types = builder.examine_h(mol_to_edit)

In [84]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
n_total_neww= atom_counts["N"]
c_total_neww= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7215
Element H: 3661
Element N: 9
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero


In [85]:
check_cc=c_total_2-(c_total_neww+math.ceil(solution[quaternaryN]))
print(check_cc)
check_n=n_total_neww-(n_total+math.ceil(solution[quaternaryN]))
print(check_n)

0
0


In [86]:
cmd.do('clean %s' % object_names)

In [87]:
builder.clear_label()

## Step 5: Create a cross-linked network to fix H/C ratio

In [88]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalN= atom_counts["N"]
FinalC1= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7215
Element H: 3661
Element N: 9
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


### Check againg cross-links by fixing H/C ratio

In [89]:
def calculate_Err_H_C(i):
    mw = FinalC1* 12.011 + i * 1.00784 + FinalN* 14.0067 + FinalO * 15.999
    H_C_mod = i/FinalC1
    Err_H_C = (abs(H_C_mod - H_C) / H_C) * 100
    return Err_H_C

# Iterate over possible values of i and find the minimum error for H/C ratio
min_error = np.inf
best_i = None
for i in range(6000):  # Adjust the range as needed based on how many carbons you have in the simulation
    error = calculate_Err_H_C(i)
    if error < min_error:
        min_error = error
        best_i = i

print("Best i value:", best_i)
print("Minimum Err_H_C:", round(min_error,3))

Best i value: 3694
Minimum Err_H_C: 0.002


#### The number of cross-links needs to be an even number

In [90]:
def round_to_lowest_even(number):
    divided_number = number / 2
    rounded_number = math.floor(divided_number)
    lowest_even_number = rounded_number * 2
    return lowest_even_number

In [91]:
Hydrogen_to_remove=round_to_lowest_even(FinalH-best_i)
print("Hydrogen to be removed:",Hydrogen_to_remove)
crosslinks=round_to_lowest_even(Hydrogen_to_remove/2)
print("Cross-links needed:",crosslinks)
newhydro=FinalH-Hydrogen_to_remove
newMW=FinalC1* 12.011 + newhydro * 1.00784 + FinalN* 14.0067 + FinalO * 15.999

Hydrogen to be removed: -34
Cross-links needed: -18


In [92]:
object_list = cmd.get_object_list()
num_objects = len(object_list)
if crosslinks >= (num_objects*0.7):
    crosslinks=round_to_lowest_even(crosslinks*0.7)
elif crosslinks <=0 or crosslinks < 20:
    crosslinks=round_to_lowest_even(FinalH*0.01)
else:
    crosslinks=crosslinks
print("Cross-links needed:",crosslinks)
newhydro=FinalH-crosslinks*2

Cross-links needed: 36


In [93]:
HydrogenMoles = (newhydro * 1.00784) / newMW
CarbonMoles = (FinalC1 * 12.011) / newMW
H_C_mod = (HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("H/C ratio:",round(H_C_mod,3))
Bridgehead_mod =1- (HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("Bridgehead Carbon:",round(Bridgehead_mod,3))

H/C ratio: 0.497
Bridgehead Carbon: 0.503


### This step is divide 4 to create a diverse distribution

In [94]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

### Step one: 50% of the total cross-links

In [95]:
excluded_prefixes = ["defect","Cdefect"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]
molecules_to_edit = random.sample(filtered_object_names, crosslinks)

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) >1 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

36


In [96]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [97]:
modified_cross = [None] * round_to_lowest_even(crosslinks/2)
mol_to_edit = [None] * (crosslinks)
h_types = [None] * (crosslinks)
h_choice = [None] * (crosslinks)

for i in range(0,  round_to_lowest_even(crosslinks/2)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round(crosslinks/2), crosslinks):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [98]:
for k in range(0, round(crosslinks/2)):
    modified_cross = builder.Crosslink_mols1(mol_to_edit[k], h_choice[k], mol_to_edit[(k+round(crosslinks/2))], h_choice[(k+round(crosslinks/2))])

In [99]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalN= atom_counts["N"]
FinalC2= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7215
Element H: 3625
Element N: 9
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero


In [100]:
Totalc=FinalC2-FinalC1
print(Totalc)

0


### Step two: + 25% of the total cross-links

In [101]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())

while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [102]:
object_names_new = list(object_masses.keys())
excluded_prefixes = ["defect","Cdefect"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]
molecules_to_edit = random.sample(filtered_object_names,round_to_lowest_even(crosslinks/2))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) >2 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

18


In [103]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [104]:
modified_cross = [None] * round_to_lowest_even(crosslinks/4)
mol_to_edit = [None] * round_to_lowest_even(crosslinks/2)
h_types = [None] * round_to_lowest_even(crosslinks/2)
h_choice = [None] *round_to_lowest_even(crosslinks/2)

for i in range(0, round_to_lowest_even(crosslinks/4)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/4), round_to_lowest_even(crosslinks/2)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [105]:
for k in range(0, round_to_lowest_even(crosslinks/4)):
    modified_cross[k]= builder.Crosslink_mols2(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/4)], h_choice[k+round_to_lowest_even(crosslinks/4)])

In [106]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalN= atom_counts["N"]
FinalC3= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7215
Element H: 3609
Element N: 9
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero

In [107]:
Totalc2=FinalC3-FinalC1
print(Totalc2)

0


### Step three: + 12.5% of the total cross-links

In [108]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [109]:
print(len(list(object_masses.keys())))

98


In [110]:
object_names_new = list(object_masses.keys())
excluded_prefixes = ["Cdefect"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]
molecules_to_edit = random.sample(filtered_object_names,round_to_lowest_even(crosslinks/4))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) > 2 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

8


In [111]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [112]:
mol_to_edit = [None] * round_to_lowest_even(crosslinks/4)
h_types = [None] * round_to_lowest_even(crosslinks/4)
h_choice = [None] *round_to_lowest_even(crosslinks/4)
for i in range(0, round_to_lowest_even(crosslinks/8)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/8), round_to_lowest_even(crosslinks/4)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [113]:
for k in range(0, round_to_lowest_even(crosslinks/8)):
    modified_cross[k]= builder.Crosslink_mols3(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/8)], h_choice[k+round_to_lowest_even(crosslinks/8)])

In [114]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalN= atom_counts["N"]
FinalC4= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7215
Element H: 3601
Element N: 9
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero

In [115]:
Totalc3=FinalC4-FinalC1
print(Totalc3)

0


### Step four: do the final 12.5% of the total cross-links

In [116]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [117]:
print(len(list(object_masses.keys())))

94


In [118]:
object_names_new = list(object_masses.keys())
excluded_prefixes = ["Cdefect"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]
molecules_to_edit = random.sample(filtered_object_names,round_to_lowest_even(crosslinks/4))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) > 2 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

8


In [119]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [120]:
mol_to_edit = [None] * round_to_lowest_even(crosslinks/4)
h_types = [None] * round_to_lowest_even(crosslinks/4)
h_choice = [None] *round_to_lowest_even(crosslinks/4)
for i in range(0, round_to_lowest_even(crosslinks/8)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/8), round_to_lowest_even(crosslinks/4)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [121]:
for k in range(0, round_to_lowest_even(crosslinks/8)):
    modified_cross[k]= builder.Crosslink_mols4(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/8)], h_choice[k+round_to_lowest_even(crosslinks/8)])

## Step 6: Check the final ultimate analysis

In [122]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [123]:
builder.clear_label()

In [124]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
Final_N= atom_counts["N"]
Final_C= atom_counts["C"]
Final_O= atom_counts["O"]
Final_H= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7215
Element H: 3593
Element N: 9
Element O: 985
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero

In [125]:
Totalc4=Final_C-FinalC1
print(Totalc4)

0


In [126]:
newMW2=Final_C* 12.011 + Final_H * 1.00784 + Final_N* 14.0067 + Final_O * 15.999
print("MW:",round(newMW2))

MW: 106166


### Ultimate Analysis

In [127]:
# Carbon
Elem_c=(Final_C*12.011)/newMW2
print("Total C:",round(Elem_c,4))
Err_elemc=(abs(Elem_c-Carbon)/Carbon)*100
print("Final Error:",round(Err_elemc,3))

Total C: 0.8163
Final Error: 0.266


In [128]:
# Nitrogen
Elem_n=(Final_N*14.0067)/newMW2
print("Total N:",round(Elem_n,5))
Err_elemn=(abs(Elem_n-Nitrogen)/Nitrogen)*100
print("Final Error:",round(Err_elemn,3))

Total N: 0.00119
Final Error: 7.945


In [129]:
# Oxygen
Elem_o=(Final_O*15.999)/newMW2
print("Total O:",round(Elem_o,3))
Err_elemo=(abs(Elem_o-Oxygen)/Oxygen)*100
print("Final Error:",round(Err_elemo,3))

Total O: 0.148
Final Error: 0.909


In [130]:
# Hydrogen
Elem_h=Final_H*1.00784/newMW2
print("Total H:",round(Elem_h,3))
Err_elemh=(abs(Elem_h-Hydrogen)/Hydrogen)*100
print("Final Error:",round(Err_elemh,3))

Total H: 0.034
Final Error: 2.547


In [131]:
# H/C ratio
CarbonMoles = (Final_C * 12.011) / newMW2
HydrogenMoles = (Final_H * 1.00784) / newMW2
H_C_mod =(HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("H/C Ratio:",round(H_C_mod,4))
Err_H_C = (abs(H_C_mod - H_C) / H_C) * 100
print("Error:",round(Err_H_C,3))

H/C Ratio: 0.498
Error: 2.736


In [132]:
# O/C ratio
OxygenMoles = (Final_O * 15.999) / newMW2
O_C_mod =1/((CarbonMoles * 15.999)/(OxygenMoles  * 12.011)) 
print("O/C Ratio:",round(O_C_mod,4))
Err_C_O = (abs(O_C_mod- O_C) / O_C) * 100
print("Error:",round(Err_C_O,3))

O/C Ratio: 0.1365
Error: 0.35


In [133]:
# Bridgehead carbon
brigd =1-(HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("Bridgehead Carbon:",round(brigd,3))
Err_brigd = (abs(brigd - BridgheadCarbon) / BridgheadCarbon) * 100
print("Error:",round(Err_brigd,3))

Bridgehead Carbon: 0.502
Error: 2.871


In [134]:
cmd.save("biochar.pse")

## Step 7: Check the final 13C NMR 

### CNMR

In [135]:
# Aromatic Carbon
Err_C=(abs(((solution[aro]-Internal_C_to_remove_Aro)/Final_C)-Aromatic)/Aromatic)*100
print("Aromatic Carbon:",round(((solution[aro]-Internal_C_to_remove_Aro)/(Final_C)),3))
print("Error:",round(Err_C,3))

Aromatic Carbon: 0.661
Error: 1.842


In [136]:
# Carbonyl Carbon
Err_Carb=(abs(((solution[carb])/Final_C)-Carbonyl)/Carbonyl)*100
print("Carbonyl Carbon:",round(((solution[carb])/(Final_C)),3))
print("Error:",round(Err_Carb,3))

Carbonyl Carbon: 0.021
Error: 9.976


In [137]:
# Carboxyl Carbon
Err_Carbox=(abs(((solution[carbox])/Final_C)-Carboxyl)/Carboxyl)*100
print("Carboxyl Carbon:",round(((solution[carbox])/(Final_C)),3))
print("Error:",round(Err_Carbox,3))

Carboxyl Carbon: 0.030
Error: 9.976


In [138]:
# Ether Carbon
Err_ethh=(abs(((solution[eth]*2)/(Final_C))-Ether)/Ether)*100
print("Ether Carbon",round((solution[eth]*2)/(Final_C),3))
print("Error:",round(Err_ethh,3))

Ether Carbon 0.111
Error: 9.976


In [139]:
# Aliphatic Carbon
Err_ali=(abs(((solution[alip])/(Final_C))-Aliphatic)/Aliphatic)*100
print("Aliphatic Carbon",round((solution[alip])/(Final_C),3))
print("Error:",round(Err_ali,3))

Aliphatic Carbon 0.033
Error: 0.027


In [140]:
# Defective Carbon
Err_def=(abs(((solution[defe])/(Final_C))-Defect)/Defect)*100
print("Defective Carbon",round((solution[defe])/(Final_C),3))
print("Error:",round(Err_def,3))

Defective Carbon 0.119
Error: 5.028


### Contribution of various C-X bonds for Aromatic and defective rings carbons (Theoretical value estimated through DFT)

### C-H 

In [141]:
# Function to count bonds between specified atoms
def count_bonds(atom1, atom2):
    selection_name = f"C-H"
    cmd.select(selection_name, f"({atom1}) within 1.8 of ({atom2})")
    bond_count = cmd.count_atoms(selection_name) - 1  # Subtract 1 to exclude the central atom
    return bond_count
    
carbon_atom = "elem C"
hydrogen_atom = "elem H"

carbon_hydrogen_bonds = count_bonds(carbon_atom, hydrogen_atom)
print(f"Number of C-H bonds: {carbon_hydrogen_bonds}")

Number of C-H bonds: 2841


In [142]:
# Take into account just bonds related to aromatic and defective carbons
print("C-H:",round((carbon_hydrogen_bonds-solution[methyl]*3-solution[ali_chain]*5-solution[ali_chain_2]*5-solution[carb]-solution[eth])/(solution[aro]+solution[defe]),3))
Err_CH = (abs((carbon_hydrogen_bonds-solution[methyl]*3-solution[ali_chain]*5-solution[ali_chain_2]*5-solution[carb]-solution[eth])/(solution[aro]+solution[defe])- C_H_)/C_H_) * 100
print("Error:",round(Err_CH,3))

C-H: 0.290
Error: 6.311


### C-O-H

In [143]:
def count_three_atom_bonds(atom1, atom2, atom3):
    selection_name = f"C-O-H"
    cmd.select(selection_name, f"(({atom1}) within 1.8 of ({atom2})) or (({atom2}) within 1.8 of ({atom3})) or (({atom1}) within 1.8 of ({atom3}))")
    bond_count = cmd.count_atoms(selection_name) - 2  # Subtract 2 to exclude the central atoms
    
    return bond_count

carbon_atom = "elem C"
oxygen_atom = "elem O"
hydrogen_atom = "elem H"

carbon_oxygen_hydrogen_bonds = count_three_atom_bonds(carbon_atom, oxygen_atom, hydrogen_atom)
print(f"Number of C–O–H bonds: {carbon_oxygen_hydrogen_bonds}")

Number of C–O–H bonds: 3690


In [144]:
# Take into account just bonds related to aromatic and defective carbons
print("C-x-H:",round((carbon_oxygen_hydrogen_bonds-solution[carb]-solution[carbox]*2-solution[eth])/(solution[aro]+solution[defe]),3))
Err_CHx = (abs((carbon_oxygen_hydrogen_bonds-solution[carb]-solution[carbox]*2-solution[eth])/(solution[aro]+solution[defe])- C_x_H_)/C_x_H_) * 100
print("Error:",round(Err_CHx,3))

C-x-H: 0.477
Error: 1.430


## Step 8: Organize the grid again and proceed to fix the helium density in LAMMPS

In [145]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [146]:
min_angle = 1
max_angle = 360.0

# Define the number of divisions in each dimension of the grid
grid_divisions = 3

# Track the positions and angles of molecules
positions = []
angles = []
xmin, xmax = 0, 90
ymin, ymax = -90, 0
zmin, zmax = 0, 90

# Shuffle the object_list to randomize the order
random.shuffle(object_names_new)

# Calculate the grid size in each dimension
grid_size_x = (xmax - xmin) / grid_divisions
grid_size_y = (ymax - ymin) / grid_divisions
grid_size_z = (zmax - zmin) / grid_divisions

# Rotate each molecule in all directions randomly
for name in object_list:
    # Generate a new position and angle until they are unique
    unique_position = False
    while not unique_position:
        # Calculate the grid indices for the position
        grid_index_x = random.randint(0, grid_divisions - 1)
        grid_index_y = random.randint(0, grid_divisions - 1)
        grid_index_z = random.randint(0, grid_divisions - 1)

        # Calculate the position within the grid cell
        x = xmin + (grid_index_x + random.uniform(0, 1)) * grid_size_x
        y = ymin + (grid_index_y + random.uniform(0, 1)) * grid_size_y
        z = zmin + (grid_index_z + random.uniform(0, 1)) * grid_size_z

        new_position = (x, y, z)

        # Check if the new position conflicts with any existing positions or angles
        if all(all(abs(new_position[i] - pos[i]) > 1e-3 for i in range(3)) for pos in positions) and new_position not in positions:
            unique_position = True
            positions.append(new_position)

            # Calculate a new random angle
            rotation_angle = random.randint(min_angle, max_angle)
            new_angle = (rotation_angle, name)

            angles.append(new_angle)

            # Perform rotation on the object
            for axis in ['x', 'y', 'z']:
                rotation_command = "rotate {0}, {1}, {2}".format(axis, rotation_angle, name)
                cmd.do(rotation_command)

# Update the display
cmd.refresh()

In [147]:
cmd.do('clean %s' % object_names_new)

In [148]:
# Obtain molecular weight distribution
object_masses = builder.Get_Object_Masses()
while not object_masses:
    object_masses = builder.Get_Object_Masses()

### Export the molecules as PDB file

In [149]:
# Select all atoms
cmd.select('all_atoms', 'all')

# Export the selection as a multi-file PDB
n_states = cmd.count_states('all_atoms')
pdb_files = []
for state in range(1, n_states + 1):
    cmd.frame(state)
    pdb_file = f'exported_molecule_{state}.pdb'
    cmd.save(pdb_file, 'all_atoms')
    pdb_files.append(pdb_file)

# Merge the multi-file PDB into a single file
merged_file = 'merged_molecule.pdb'
with open(merged_file, 'w') as outfile:
    for pdb_file in pdb_files:
        with open(pdb_file, 'r') as infile:
            outfile.write(infile.read())

# Save the merged molecule
cmd.load(merged_file, 'merged_molecule')
cmd.save('final_molecule.pdb', 'merged_molecule')

# Clean up the intermediate files
for pdb_file in pdb_files:
    os.remove(pdb_file)
os.remove(merged_file)

In [150]:
# Call the PyMOL aliases
cmd.do('colour')
cmd.do('orient')
# Change colors ans appereance
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')

## Step 9: Convert the PDB file to Lammps data using OVITO

#### To define the x, y, z values use the following code:

In [151]:
HeliumDensity = float(input("Enter the value for Helium Density: "))  # kg/m3

Enter the value for Helium Density:  1.470


In [152]:
Temperature=298                             # K
Pressure=1                                  # atm
Density=(HeliumDensity*1000)/(10**(27))     # g/Å3
NA=6.022*10**(23)                           # molecules per mol
BoxVolume=1/((Density/newMW2)*NA)           # Å3
BoxLength=(BoxVolume)**(1/3)                # Å
print(" x, y, z:",round(BoxLength,3))


 x, y, z: 49.315


#### Open the PDB file in VMD, and in the console put the following commands:

cd Desktop

pbc set {x y z} -all


pbc box -center all

topo writelammpsdata biochar.data charge

You can also convert it directly using OVITO, just open the file and export it as LAMMPS data using charge